# Reading data from datasets

In [1]:
import util
import numpy as np
import random

In [2]:
# Available datasets: p01 to p08, c08 to c11
[[capacity], weights, profits, optimal_solution] = util.read_data('p01')
optimal_profits = np.multiply(optimal_solution, profits).sum()

In [3]:
capacity

165

In [4]:
weights

[23, 31, 29, 44, 53, 38, 63, 85, 89, 82]

In [5]:
profits

[92, 57, 49, 68, 60, 43, 67, 84, 87, 72]

In [6]:
optimal_solution

[1, 1, 1, 1, 0, 1, 0, 0, 0, 0]

In [7]:
optimal_profits

309

# Actual Computation

In [8]:
def generate_candidates(solution_length, num_of_candidates):
    return [util.join_to_str([1 if random.uniform(0, 1) >= 0.5 else 0 for i in range(solution_length)]) for j in range(num_of_candidates)]

In [9]:
def knapsack_genetic(c, w, p):
    # create first generation with 10 candidates
    candidates = generate_candidates(len(w), 10)
    scores = [np.multiply(util.split_to_num(candidate), profits).sum() for candidate in candidates]
    print(scores)
    return '', ''

In [10]:
computed_profits, s = knapsack_genetic(capacity, weights, profits)
# computed_solution = util.split_to_num(s)

[403, 252, 286, 343, 216, 301, 470, 466, 395, 349]


In [11]:
# print(f'optiomal profits: {optimal_profits}')
# print(f'computed_profits: {computed_profits}')

In [12]:
# print(f'optimal_solution: \n{optimal_solution}')
# print(f'computed_solution: \n{computed_solution}')

In [13]:
random.uniform(0, 1)

0.7469585139841368

In [14]:
''.join(str(i) for i in [0, 1, 2])

'012'